In [1]:
"""
Some results (e.g. miconic evaluation, or blocksworld training) are either timing out (due to apptainer and cluster cpus) or running out of memory (possibly due to suboptimal handcrafted policies, or very large training datasets).
"""

from _results_helper import *

this_file_dir=PosixPath('/home/dzc/code/cvut-colab/results')
Raw logs found. Rewriting data to csv files...


/home/dzc/code/cvut-colab/results/baseline_logs/transport_1_30_r9.log: 100%|██████████| 2400/2400 [00:00<00:00, 4079.38it/s] 


skipping hard problems because they take too long


# Plan length plots

In [2]:
log_y = 0
include_sample = 0
layers = [
    1,
    2,
    3,
    # 4,
    # 5,
]
dimensions = [
    1,
    # 2,
    # 4,
    # 8,
    16,
    32,
    64,
]

plot_domains("plan_length", log_y=log_y, include_sample=include_sample, layers=layers, dimensions=dimensions)

blocksworld


ferry


satellite


transport


In [3]:
# plot_domains("time")

# Absolute difference plots

In [4]:
include_sample = 0

plot_difference(absolute=True, include_sample=include_sample, layers=layers, dimensions=dimensions)

blocksworld


ferry


satellite


IndexError: index 0 is out of bounds for axis 0 with size 0

# Relative difference plots

In [ ]:
include_sample = 0
plot_difference(absolute=False, include_sample=include_sample, layers=layers, dimensions=dimensions)

blocksworld


ferry


satellite


transport
